# 🚀 Get started to validate the setup

This Jupyter notebook is recommended for workshop/education only.

Prerequisites:

1. Set up your computing environment
2. Install the required library in your Python environment
3. Select the correct kernel (`azureml_py310_sdkv2`) for your Jupyter notebook


## 1. Azure Open AI Test

---


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = "../../0_lab_preparation"
sys.path.append(os.path.abspath(module_path))

from common import check_kernel
check_kernel()

In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")
    
try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version
    )
    deployment_name = aoai_deployment_name
except (ValueError, TypeError) as e:
    print(e)

In [3]:
# Create your prompt
system_message = """
あなたは顧客が情報を見つけるのを助けるAIアシスタントです。
アシスタントとして簡潔、簡潔、そしてパーソナライズされた方法で質問に答えること。
また、マークダウンを使用して、回答をシンプルかつ簡潔に保ち、適切な絵文字で個人的なタッチを追加することもできます。

答えの最後に「ところで」で始まる機知に富んだジョークを追加します。ジョークの部分では、回答の部分に顧客の名前を記載しないでください。
ジョークは、尋ねられる特定の質問に関連している必要があります。
たとえば、質問がテントに関するものである場合、ジョークは特にテントに関連するものであるべきです。

次のコンテキストを使用して、よりパーソナライズされた応答を提供します。各文の行を変更し、日本語で回答してください。
"""

In [ ]:
context = """
    アルパインエクスプローラーテントには、プライバシーを確保するために取り外し可能な仕切りがあります。
    多数のメッシュ窓と換気用の調節可能な通気口、および
    防水設計が特徴です。アウトドアの必需品を収納するためのギアロフトも組み込まれています
    機器ロフトが組み込まれています。要するに、プライバシー、快適さ、
    便利さの組み合わせは、自然の中でそれを第二の家にします!
"""
question = "アルパインエクスプローラーテントの特徴は何ですか?"

user_message = f"""
Context: {context}
Question: {question}
"""

# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
  temperature=0.7,
  max_tokens=300
)

print(response.choices[0].message.content)

## (Optional) 2. Azure Document Inteligence Test

---


In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat

doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

try:
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )
except (ValueError, TypeError) as e:
    print(e)
    
raw_data_dir = "../1_synthetic-qa-generation/raw_data"
file_path = f"{raw_data_dir}/pdf/en-imagenet-training-wrote-by-daekeun.pdf"

In [ ]:
with open(file_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
        output_content_format=ContentFormat.MARKDOWN 
    )

result = poller.result()
md_content = result.content
print(md_content)

## 3. Azure ML Test

---


In [ ]:
import os
import yaml
from datetime import datetime
snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open('../2_slm-fine-tuning-mlstudio/phi3/config.yml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_DATA_NAME']    
DATA_DIR = d['config']['DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']
IS_DEBUG = d['config']['IS_DEBUG']
USE_LOWPRIORITY_VM = d['config']['USE_LOWPRIORITY_VM']


print(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
print(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
print(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
print(f"AZURE_DATA_NAME={AZURE_DATA_NAME}")
print(f"DATA_DIR={DATA_DIR}")
print(f"CLOUD_DIR={CLOUD_DIR}")
print(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
print(f"IS_DEBUG={IS_DEBUG}")
print(f"USE_LOWPRIORITY_VM={USE_LOWPRIORITY_VM}")


In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

# from azure.identity import ClientSecretCredential
# credentials = ClientSecretCredential(
#     client_id=client_id,
#     client_secret=client_secret,
#     tenant_id=tenant_id
# )

try:
    workspace = ml_client.workspaces.get(name=AZURE_WORKSPACE)
    print(f"Connected to Azure ML Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")